In [15]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime, timedelta
import time
import random
import os

In [11]:
def generate_date_list(start, end):
    date_list = []
    curr = start
    while curr <= end:
        date_list.append(curr.strftime("%Y.%m.%d"))
        curr += timedelta(days=1)
    return date_list

In [5]:
def extract_urls(json_data):
    urls = []
    for item in json_data.get('collection', []):
        html_str = item.get('html', '')
        soup = BeautifulSoup(html_str, 'html.parser')
        links = soup.find_all('a', href=True)
        for link in links:
            if 'n.news.naver.com' in link['href']:
                urls.append(link['href'])
    unique_urls = list(set(urls))
    return unique_urls

In [27]:
def get_urls_bydate(date, office_id):
    collected_urls = []
    base_url = "https://s.search.naver.com/p/newssearch/3/api/tab/more"
    headers = {'User-Agent': 'Mozilla/5.0', 'Referer': 'https://search.naver.com/'}
    for start in range(1, 2000, 10):
        params = {
        'abt': 'null',
        'de': date,
        'ds': date,
        'field': '0',
        'is_dts': '0',
        'is_sug_officeid': '0',
        'mynews': '1',
        'news_office_checked': office_id,
        'nqx_theme': '{"theme":{"sub":[{"name":"finance"}]}}',
        'nso': f'so:r,p:from{date.replace(".","")}to{date.replace(".","")},a:all',
        'office_category': '0',
        'office_section_code': '3',
        'office_type': '1',
        'pd': '3',
        'photo': '0',
        'query': '금리',
        'rev': '0',
        'service_area': '0',
        'sm': 'tab_smr',
        'sort': '2',
        'spq': '0',
        'ssc': 'tab.news.all',
        'start': start
        }
        try:
            res = requests.get(base_url, headers=headers, params=params, timeout=10)
            if res.status_code != 200 or not res.text.strip():
                print(f"[{date}] 수집 종료")
                break
            data = res.json()
            page_urls = extract_urls(data)
            
            if not page_urls:
                break
            
            collected_urls.extend(page_urls)
            
            time.sleep(random.uniform(0.3, 0.6))
            
        except Exception as e:
            print(f"[{date}] 에러 발생: {e}")
            raise e
            
    return list(set(collected_urls))

In [28]:
offices = {"매일경제": "1009", "한국경제": "1015", "머니투데이": "1008"}
keyword = "금리"
s_dt = datetime(2005, 5, 1) 
e_dt = datetime(2005, 5, 30)
max_workers = 10

In [29]:
office_id = offices['매일경제']
file_name = f'news_urls_{office_id}.csv'
target_dates = generate_date_list(s_dt, e_dt)
print(f"총 {len(target_dates)}일치 수집을 시작합니다.")

for i, date in enumerate(target_dates):
    print(f"\n[진행도: {i+1}/{len(target_dates)}] {date} 수집 중...")
    urls = get_urls_bydate(date, office_id)

    if urls:
        df = pd.DataFrame(urls, columns=['url'])
        df['date'] = date
        if not os.path.exists(file_name):
            df.to_csv(file_name, index=False, encoding='utf-8-sig')
        else:
            df.to_csv(file_name, index=False, encoding='utf-8-sig', mode='a', header=False)
        print(f"{date} 저장 완료({len(urls)}개 추가됨)")
    else:
        print(f"{date}는 검색 결과 없음. 다음 날로 넘어갑니다.")

print(f"\n모든 수집 완료. '{file_name}' 파일을 확인해 보세요.")


총 30일치 수집을 시작합니다.

[진행도: 1/30] 2005.05.01 수집 중...
2005.05.01 저장 완료(8개 추가됨)

[진행도: 2/30] 2005.05.02 수집 중...
2005.05.02 저장 완료(8개 추가됨)

[진행도: 3/30] 2005.05.03 수집 중...
2005.05.03 저장 완료(6개 추가됨)

[진행도: 4/30] 2005.05.04 수집 중...
2005.05.04 저장 완료(8개 추가됨)

[진행도: 5/30] 2005.05.05 수집 중...
2005.05.05 저장 완료(3개 추가됨)

[진행도: 6/30] 2005.05.06 수집 중...
2005.05.06 저장 완료(6개 추가됨)

[진행도: 7/30] 2005.05.07 수집 중...
2005.05.07는 검색 결과 없음. 다음 날로 넘어갑니다.

[진행도: 8/30] 2005.05.08 수집 중...
2005.05.08 저장 완료(5개 추가됨)

[진행도: 9/30] 2005.05.09 수집 중...
2005.05.09 저장 완료(13개 추가됨)

[진행도: 10/30] 2005.05.10 수집 중...
2005.05.10 저장 완료(7개 추가됨)

[진행도: 11/30] 2005.05.11 수집 중...
2005.05.11 저장 완료(12개 추가됨)

[진행도: 12/30] 2005.05.12 수집 중...
2005.05.12 저장 완료(10개 추가됨)

[진행도: 13/30] 2005.05.13 수집 중...
2005.05.13 저장 완료(3개 추가됨)

[진행도: 14/30] 2005.05.14 수집 중...
2005.05.14 저장 완료(1개 추가됨)

[진행도: 15/30] 2005.05.15 수집 중...
2005.05.15 저장 완료(6개 추가됨)

[진행도: 16/30] 2005.05.16 수집 중...
2005.05.16 저장 완료(8개 추가됨)

[진행도: 17/30] 2005.05.17 수집 중...
2005.05.17 저장 완료(

In [ ]:
# 수집 후 겹치는 url 지우기
df = pd.read_csv(f'news_urls_{office_id}.csv')
df_clean = df.drop_duplicates(subset=['url'], keep='first')
df_clean.to_csv(f"news_urls_{office_id}_final.csv", index=False, encoding='utf-8-sig')

print(f"정리 전: {len(df)}개 -> 정리 후: {len(df_clean)}개")

정리 전: 570개 -> 정리 후: 212개


In [ ]:
# 매일경제 뉴스 수집
def mail_news_detail(url):
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'Referer': 'https://news.naver.com/'
    }
    try:
        res = requests.get(url, headers=headers, timeout=15)
        if res.status_code != 200:
            return None
        soup = BeautifulSoup(res.text, 'html.parser')
        
        title = soup.select_one("h2#title_area")
        title = title.get_text(strip=True) if title else "제목 없음"
        
        date_tag = soup.select_one("span.media_end_head_info_datestamp_time")
        if date_tag and date_tag.has_attr('data-date-time'):
            date_str = date_tag['data-date-time']
        else:
            date_str = date_tag.get_text(strip=True) if date_tag else "날짜 없음"
        content = soup.select_one("#newsct_article") # 또는 "#dic_area"
        
        if content:
            content = content.get_text(" ", strip=True)
        else:
            content = "본문 없음"
            
        return {'title': title, 'date': date_str, 'content': content, 'url': url}

    except Exception as e:
        print(f"\n[{url}] 수집 중 에러 발생: {e}") 
        raise e

news_detail('https://n.news.naver.com/mnews/article/009/0000437454?sid=101')

{'title': '[금융 신상품] 국고채 年 3.70~3.85%',
 'date': '2005-05-01 13:17:00',
 'content': '지난주 금리는 하락세를 보였다. 5월 국채발행 계획, 3월 산업활동 결과가 시 장 예상 수준을 벗어나지 않았고 미국 1분기 GDP 성장률이 부진하면서 소프트 패치 논쟁이 가열됐기 때문이다. 3월 산업활동 동향은 분명 국내 경기가 저점을 지나고 있음을 확인시켜 주고 있다. 그러나 지표개선 속도가 완만하며 미국 경제지표 부진은 2분기 이후 국 내 경기 불확실성을 키우는 요인이다. 글로벌 경기 둔화 염려가 부각되는 상황 에서 위안화 절상 가능성에 따른 원화 절상 압력은 경기 측면에서 금리하락 요 인으로 해석될 가능성이 높다. 물론 금리 수준 부담으로 금리 추가 하락 여지는 좁아 보이지만 금리 상승 압 력 역시 미약할 것 같다. 이번주 지표금리(3년만기 국고채 수익률)는 연 3.70~ 3.85%에서 움직일 전망이다. [김형기 대우증권 선임연구원] < Copyright ⓒ 매일경제. 무단전재 및 재배포 금지 >',
 'url': 'https://n.news.naver.com/mnews/article/009/0000437454?sid=101'}